In [2]:
# Importing modules
import folium
from pyproj import crs
import geopandas as gpd
import matplotlib.pyplot as plt

In [3]:
# Create a Map instance
m = folium.Map(location=[60.25, 24.8], zoom_start=10, control_scale=True)